In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/1.year_code.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Vehicle Type Code'] = input_['Vehicle Type Code'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(f'''https://www.moogparts.com/driv/partfinder/api.catalog.makes?brand=moog&locale=en_US&country_code=US&year_id={input_.loc[a, 'Year Code']}&vehicle_group_ids={input_.loc[a, 'Vehicle Type Code']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))
                    
                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_make_code = [dict_['id'] for dict_ in json_['makes']]
                        if list_make_code:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Vehicle Type Code': input_.loc[a, 'Vehicle Type Code'],
                                    'Make Code': list_make_code,
                                    'Year Code': input_.loc[a, 'Year Code']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Vehicle Type Code']} > {input_.loc[a, 'Year Code']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Vehicle Type Code', 'Make Code', 'Year Code'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/2.make_code-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Vehicle Type Code', 'Year Code'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/make_code_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：154

[状态：ok，尝试次数：7] - 2 > 1960
[剩余数量：124] - [当前时间：15:49:34]

[状态：ok，尝试次数：3] - 2 > 1963
[剩余数量：123] - [当前时间：15:49:34]

[状态：ok，尝试次数：11] - 2 > 1950
[剩余数量：122] - [当前时间：15:49:34]

[状态：ok，尝试次数：12] - 2 > 1941
[剩余数量：121] - [当前时间：15:49:35]

[状态：ok，尝试次数：17] - 2 > 1948
[剩余数量：120] - [当前时间：15:49:35]

[状态：ok，尝试次数：8] - 2 > 1955
[剩余数量：119] - [当前时间：15:49:35]

[状态：ok，尝试次数：23] - 2 > 1951
[剩余数量：118] - [当前时间：15:49:35]

[状态：ok，尝试次数：21] - 2 > 1940
[剩余数量：117] - [当前时间：15:49:35]

[状态：ok，尝试次数：15] - 2 > 1964
[剩余数量：116] - [当前时间：15:49:35]

[状态：ok，尝试次数：20] - 2 > 1959
[剩余数量：115] - [当前时间：15:49:35]

[状态：ok，尝试次数：10] - 2 > 1949
[剩余数量：114] - [当前时间：15:49:36]

[状态：ok，尝试次数：34] - 2 > 1958
[剩余数量：113] - [当前时间：15:49:36]

[状态：ok，尝试次数：37] - 2 > 1952
[剩余数量：112] - [当前时间：15:49:37]

[状态：ok，尝试次数：5] - 2 > 1967
[剩余数量：111] - [当前时间：15:49:37]

[状态：ok，尝试次数：44] - 2 > 1962
[剩余数量：110] - [当前时间：15:49:37]

[状态：ok，尝试次数：49] - 2 > 1946
[剩余数量：109] - [当前时间：15:49:38]

[状态：ok，尝试次数：54] - 2 > 1945
[剩余数量：108] - [当前时间：15:49:38]

[状态：ok，尝试次数：2] - 2 > 1977
